<a href="https://colab.research.google.com/github/drjraclarke/ISO-3166-Countries-with-Regional-Codes/blob/master/splicing_tool_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **In silico deep mutational scanning with Splice AI v. 1.2 **
**This end-to-end tool visualises the predicted regulatory and modulatory architecture of alternative exon splicing for exons and neighbouring intronic regions queried by you.** It performs in silico deep mutational scanning using [SPLICE AI (Jaganathan et al)](https://doi.org/10.1016/j.cell.2018.12.015), a deep convolutional neural network (CNN) that predicts splice junctions from primary pre-mRNA transcripts with high accuracy. Splice AI DIM handles the genomic sequence queries, indel library preparation, running of the CNN and performs downstream analyses for you using a Google colabotory hosted GPU. Google colaboratory free subscriptions do have limited memory and runtime which can vary with community usage.

The output is visualised as a series of heatmaps of all deletions and substituions, and a curve fitted to 1-6nt length deletions. There are options to: change the nucelotide windown splice AI scans either side of each variant, vary the size of the indel library (and resultant plots) and vary the number of nucelotides in 5' and 3' intronic regions that are included in the analysis.

**How to use**
1.   Log into google account
2.   Select "Runtime" > "Change run time type" and select any available option with "GPU" or "TPU".
3.   Build query from options in below panel. Default options produce optimal plotting. Advised to select download option so model output is saved once complete in case connection becomes idle. Ensembl exon ID not needed if gene and exon number known.
3.   Run all cells ("Runtime" > "Run all")
5.   Visualise output in notebook and/or locally on your machine (if download option selected for)

**FAQs/troubleshooting**

*   Make sure your gene name and exon number correspond to the Ensembl database naming. If in doubt or errors arise, search the Ensembl database for your gene, navigate to the primary transcript, find the list of exons and see that your query aligns with their exon ranking. This tool queries Ensembl for you but does require precise name and exon matching with no syntax errors. Note Ensembl does not take exon numerical_alphabateical names as input (eg. Exon 2B).
*   The compute time will increase substantially (from approx 5-10 mins to 60-100 mins) with increasing size of the indel library used as input to SpliceAI. Indel library size is determined by the customisable options and expansion of 3 & 5 prime ends of the exon. Caution when substantially expanding the intronic regions, this may mean the neighbouring exon is included. This can be checked at Ensembl by knowing the size of the introns in question.
*   The output files will be named according to your specific query. They will be in your downloads folder. There may be a google notification to allow mutliple downloads which requires your agreement. They will include a query csv file detailing the options you selected.
*   Any feedback, queries or problems please do contact Joe Clarke at jc58@sanger.ac.uk. Ideally use the subject header "SpliceAI tool query".  
*   If the code and output from each cell are visible and you wish to hide them, you can do so by double clikcing on the title of the cell. Alternatively, "Edit" > "Select all cells" > "View" > "Show/hide code" > "Show/hide output".
*   No recall thresholds are assigned in this tool for the Splice AI output. For a detailed discussion regarding thresholds and guidance on the precision/recall trade-offs, read the publication by Jaganathan et al.





*With thanks to Pablo Baeza-Centurion,Belen Minana, Andre J. Faure, Mike Thompson, Sophie Bonnal,Gioia Quarantini, Juan Valcarcel and Ben Lehner at the Centre for Genomic Regulation (Barcelona, Spain) and Wellcome Sanger Institute (Cambridge, UK) for developing the methods behind this tool. Thanks to the team at SPLICE AI and [SPLICE AI VISUAL](https://doi.org/10.1186/s40246-023-00451-1)/BATCH SPLICE for their research and existing colaboratory notebook which was used as a guide for this tool development.*















In [ ]:
import csv
GRCh = "38"  #@param [37, 38]
if GRCh == "37":
  hg = "19"
if GRCh == "38":
  hg = GRCh

window =  125#@param {type:"integer"}
#@markdown  * Must be between 50-4999 (max distance between the variant and gained/lost splice site predicted by SpliceAI (default: 50). <br/>
automatic_download = True #@param {type:"boolean"}
#@markdown  * `GRCh` : 37 or 38 <br/>
#@markdown * `window` : must be between `50` et `4999`
#@markdown * `automatic_download` : <img src="https://www.tensorflow.org/images/download_logo_32px.png" width=20> automatically
#@markdown download the plots and metadata when finished
if window > 4999:
  print("ERROR : window for splice AI analysis must be between 50 and 4999")

  # Text input for "Ensembl Gene Name"
ensembl_gene_name_query = "ST7" #@param {type:"string"}

# Input for "Exon Number"
exon_number_query = 7 #@param {type:"integer"}

# Optional input for ensembl exon ID if already known
ensembl_exon_id = ""#@param {type:"string"}
#@markdown  * not necessary if gene and exon number known <br/>
# Indel library options
default_deletion_sizes  = True #@param {type:"boolean"}
#@markdown  * Default deletion lengths = (1-6nt, 18nt, 21nt) <br/>
custom_deletion_sizes  = "1,2,3,4,5,6,7,8,9,10,21" #@param {type:"string"}
#@markdown  * Separate by a comma eg. "1,3,5". Note, 1-6nt deletions (1,2,3,4,5,6) minimum required for plotting. <br/>
include_all_possible_substitutions = True #@param {type: "boolean"}

#Input for "Intronic_region_3prime"
expand_3prime = 25 #@param {type:"integer"}

#Input fpr "Intronic_region_5prime"
expand_5prime = 70 #@param {type:"integer"}
# Display the values
print(f"Ensembl Gene Name: {ensembl_gene_name_query}")
print(f"Exon Number: {exon_number_query}")
print(f"Ensembl exon ID : {ensembl_exon_id}")
print(f"Intronic 3 prime inclusion : {expand_3prime} nucleotides")
print(f"Intronic 5 prime inclusion : {expand_5prime} nucelotides")
print("Indel library:")
if default_deletion_sizes :
    print(" - Deletions: 1-6nt, 18nt, 21nt")
else:
    # Assuming custom_deletion_sizes is a string of numbers separated by commas
    print(f" - Deletions: {custom_deletion_sizes} nt")

if include_all_possible_substitutions:
    print(" - Substitutions: Yes")
else:
    print(" - Substitutions: No")

deletion_sizes = "1,2,3,4,5,6,18,21" if default_deletion_sizes else custom_deletion_sizes

# Define the headers and the row of data
headers = ["Ensembl Gene Name", "Exon Number","Ensembl Exon ID", "Intronic 3 prime inclusion", "Intronic 5 prime inclusion", "Deletions", "Substitutions"]
data_row = [
    ensembl_gene_name_query,
    exon_number_query,
    f"{expand_3prime} nt",
    f"{expand_5prime} nt",
    deletion_sizes,
    "Yes" if include_all_possible_substitutions else "No"
]

# Write to a CSV file
query_csv_file_path = f"{ensembl_gene_name_query}_{exon_number_query}_query.csv"
with open(query_csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(headers)
    writer.writerow(data_row)

In [ ]:
#@title Install required libraries, modules and reference files. Load functions.
# Install required libraries
%pip install biopython
%pip install pyfastx
%pip install tensorflow
%pip install spliceai
%pip install ensembl_rest

# Import the necessary modules
import ensembl_rest
import pandas as pd
import requests
import sys
from Bio.Seq import Seq
import pyfastx
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

!wget -nc http://hgdownload.soe.ucsc.edu/goldenPath/hg{hg}/bigZips/hg{hg}.fa.gz
!gzip -df hg{hg}.fa.gz
hgfastafile = pyfastx.Fasta('hg{0}.fa'.format(hg))




In [ ]:
#@title Load functions
def generate_all_singles(wt_sequence, exon_id, chromosome, padded_chrom_start):
    wt_sequence_list = list(wt_sequence)
    my_singles = []
    my_ids = []
    wt_sequences = []  # To store the WT sequence for each variant

    for i, nucleotide in enumerate(wt_sequence_list):
        possible_substitutions = [nt for nt in ["A", "T", "G", "C"] if nt != nucleotide]
        for each_substitution in possible_substitutions:
            new_single_list = wt_sequence_list.copy()
            new_single_list[i] = each_substitution
            new_single = "".join(new_single_list)
            my_ids.append(f"chr{chromosome}:{padded_chrom_start}_ex{exon_id}_Sub_{nucleotide}{i + 1}{each_substitution}")
            my_singles.append(new_single)
            wt_sequences.append(wt_sequence)  # Add the WT sequence

    return pd.DataFrame({
        'ID': my_ids,
        'Sequence': my_singles,
        'Chromosome': [chromosome] * len(my_ids),
        'WT_sequence': wt_sequences
    })

def generate_all_deletions(k, WT, exon_id, chromosome, padded_chrom_start):
    WT_vectorized = list(WT)
    number_of_deletions = len(WT_vectorized) - k + 1
    sequences = []
    IDs = []
    wt_sequences = []  # To store the WT sequence for each variant

    for i in range(number_of_deletions):
        deleted_positions = range(i, i + k)
        sequence_with_deletion = ''.join([WT_vectorized[j] for j in range(len(WT_vectorized)) if j not in deleted_positions])
        ID = f"chr{chromosome}:{padded_chrom_start}_ex{exon_id}_Del_k{k}_{i + 1}-{i + k}"
        sequences.append(sequence_with_deletion)
        IDs.append(ID)
        wt_sequences.append(WT)  # Add the WT sequence

    return pd.DataFrame({
        'ID': IDs,
        'Sequence': sequences,
        'Chromosome': [chromosome] * len(IDs),
        'WT_sequence': wt_sequences
    })

def remove_duplicates_in_variant_data_frame(variant_data_frame):
    unique_sequences = {}

    # Adjusting the loop to also iterate over the WT_sequence column
    for ID, sequence, chromosome, WT_sequence in zip(variant_data_frame['ID'], variant_data_frame['Sequence'], variant_data_frame['Chromosome'], variant_data_frame['WT_sequence']):
        unique_key = (sequence, chromosome, WT_sequence)  # Adding WT_sequence to the unique key
        if unique_key not in unique_sequences:
            unique_sequences[unique_key] = {'IDs': [ID], 'Chromosome': chromosome, 'WT_sequence': WT_sequence}
        else:
            unique_sequences[unique_key]['IDs'].append(ID)

    # Adjusting the structure of the new_data_frame to include WT_sequence
    new_data_frame = {'Sequence': [], 'ID': [], 'Chromosome': [], 'WT_sequence': []}

    for (sequence, _, WT_sequence), details in unique_sequences.items():
        new_data_frame['Sequence'].append(sequence)
        new_data_frame['ID'].append(';'.join(details['IDs']))
        new_data_frame['Chromosome'].append(details['Chromosome'])
        new_data_frame['WT_sequence'].append(details['WT_sequence'])  # Adding WT_sequence to each row

    return pd.DataFrame(new_data_frame)

# Define a function to calculate exon_start and exon_end
def calculate_exon_positions(row):
    if row['strand_orientation'] == -1:
        exon_start = row['padded_chrom_start'] + (expand_3prime+1)
        exon_end = row['padded_chrom_end'] - (expand_5prime+1)
    else:
        exon_start = row['padded_chrom_start'] + (expand_5prime+1)
        exon_end = row['padded_chrom_end'] - (expand_3prime+1)
    return pd.Series([exon_start, exon_end], index=['exon_start', 'exon_end'])

    server = "https://rest.ensembl.org"
def get_canonical_transcript_id(gene_name):
    # Base URL for the Ensembl REST API
    server = "https://rest.ensembl.org"

    # Endpoint for lookup by gene symbol
    ext_lookup = f"/lookup/symbol/homo_sapiens/{ensembl_gene_name_query}?expand=1"

    # Make the request to get gene data
    response = requests.get(server + ext_lookup, headers={"Content-Type": "application/json"})

    if not response.ok:
        response.raise_for_status()
        sys.exit()

    decoded = response.json()

    # Extract the canonical transcript ID
    for transcript in decoded.get('Transcript', []):
        if transcript.get('is_canonical', 0) == 1:
            return transcript['id']
    return None

def get_exon_id_list(transcript_id: str) -> list[str]:
    tr_details = _fetch_feature_details(transcript_id)
    exon_list = tr_details.get('Exon')
    return [e.get('id') for e in exon_list]

def _fetch_feature_details(stable_id: str) -> dict:
    ext = f"/lookup/id/{stable_id}?expand=1"
    return _get_from_REST(ext)

def _get_from_REST(endpoint: str) -> dict:
    r = requests.get(server+endpoint, headers={"Content-Type": "application/json"})
    if not r.ok:
        r.raise_for_status()
    return r.json()

def get_specific_exon_id(transcript_id: str, exon_number_query: int) -> str:
    exon_ids = get_exon_id_list(transcript_id)

    # Adjust for zero-based indexing and check if the index is in range
    if 0 < exon_number_query <= len(exon_ids):
        return exon_ids[exon_number_query - 1]
    else:
        return None




In [ ]:
# @title Query Ensembl

server = "https://rest.ensembl.org"
if ensembl_exon_id == "":
     canonical_transcript_id = get_canonical_transcript_id(ensembl_gene_name_query)
     ensembl_exon_id = get_specific_exon_id(canonical_transcript_id, exon_number_query)
else:
  ensembl_exon_id == ensembl_exon_id
def get_exon_info(exon_id, expand_3prime, expand_5prime):
    ext = f"/sequence/id/{exon_id}?type=genomic;expand_3prime={expand_3prime+1};expand_5prime={expand_5prime+1}"
    response = requests.get(server + ext, headers={"Content-Type": "text/x-fasta"})

    if not response.ok:
        response.raise_for_status()

    fasta_data = response.text
    header_line, sequence = fasta_data.split('\n', 1)[0], ''.join(fasta_data.split('\n')[1:])

    # Extract metadata from the header line
    parts = header_line.split(':')
    chromosome, padded_chrom_start, padded_chrom_end, strand_orientation = parts[2], int(parts[3]), int(parts[4]), int(parts[5])


    return {
        'exon_id': exon_id,
        'chromosome': chromosome,
        'padded_chrom_start': padded_chrom_start,
        'padded_chrom_end': padded_chrom_end,
        'strand_orientation': strand_orientation,
        'padded_exon_sequence': sequence
    }
exon_info = get_exon_info(ensembl_exon_id, expand_3prime, expand_5prime)
exon_info_df = pd.DataFrame([exon_info])

# Apply the function to calculate exon positions
exon_info_df[['exon_start', 'exon_end']] = exon_info_df.apply(calculate_exon_positions, axis=1)

import pandas as pd

# Assuming exon_info_df is already prepared with 'exon_start' and 'exon_end' columns
# Initialize an empty DataFrame to store position lookup information for all exons
all_position_lookups = pd.DataFrame()

for index, row in exon_info_df.iterrows():
    exon_id = row['exon_id']
    padded_exon_sequence = row['padded_exon_sequence']
    chromosome = row['chromosome']
    padded_chrom_start = row['padded_chrom_start']
    padded_chrom_end = row['padded_chrom_end']
    exon_start = row['exon_start']  # Assuming this column exists
    exon_end = row['exon_end']  # Assuming this column exists

    positions = list(range(1, len(padded_exon_sequence) + 1))
    nucleotides = list(padded_exon_sequence)
    chromosomes = [chromosome] * len(positions)

    # Calculate reference positions
    reference_positions = list(range(padded_chrom_start, padded_chrom_start + len(padded_exon_sequence))) if row['strand_orientation'] != -1 else list(range(padded_chrom_end, padded_chrom_end - len(padded_exon_sequence), -1))[::-1]

    # Determine regions (EXON or INTRON)
    regions = ['EXON' if exon_start <= pos <= exon_end else 'INTRON' for pos in reference_positions]

    # Create DataFrame for current exon
    position_lookup = pd.DataFrame({
        'position': positions,  # Starting from 1
        'nucleotide': nucleotides,
        'chromosome': chromosomes,
        'reference position': reference_positions,
        'exon ID': [exon_id] * len(positions),
        'region': regions  # Add region based on the condition
    })

    # Append to the all_position_lookups DataFrame
    all_position_lookups = pd.concat([all_position_lookups, position_lookup], ignore_index=True)

# Now all_position_lookups contains position lookup information for all exons, with indexing starting from 1
# Optionally, save this DataFrame to a CSV file
all_position_lookups.index = all_position_lookups.index + 1  # Add 1 to the index
#all_position_lookups
# Define the headers and the row of data
headers = ["Ensembl Gene Name", "Exon Number","Ensembl Exon ID", "Intronic 3 prime inclusion", "Intronic 5 prime inclusion", "Deletions", "Substitutions"]
data_row = [
    ensembl_gene_name_query,
    exon_number_query,
    f"{expand_3prime} nt",
    f"{expand_5prime} nt",
    deletion_sizes,
    "Yes" if include_all_possible_substitutions else "No"
]

# Write to a CSV file
query_csv_file_path = f"{ensembl_gene_name_query}_{exon_number_query}_{ensembl_exon_id}_query.csv"
with open(query_csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(headers)
    writer.writerow(data_row)

exon_info_df

In [ ]:
# @title Create indel library and encode as VCF file for input to splice AI
import re
import pandas as pd

import pandas as pd
from Bio.Seq import Seq
import re

# Assuming exon_info_df and deletion_sizes are already defined

# Combined DataFrame initialization
combined_variants_df = pd.DataFrame()

# Iterate over each exon
for index, row in exon_info_df.iterrows():
    exon_id = row['exon_id']
    sequence = row['padded_exon_sequence']
    strand_orientation = row['strand_orientation']
    chromosome = row['chromosome']
    padded_chrom_start = row['padded_chrom_start']  # Extract padded_chrom_start from the row

    # Determine the WT sequence based on strand orientation
    WT = str(Seq(sequence).reverse_complement()) if strand_orientation == -1 else sequence

if include_all_possible_substitutions:
    # Generate all single nucleotide substitutions
    singles_df = generate_all_singles(WT, exon_id, chromosome, padded_chrom_start)
    singles_df['Exon_id'] = exon_id  # Add the exon_id to each substitution
    singles_df['Chromosome'] = chromosome  # Add the chromosome number to each substitution
    singles_df['WT_sequence'] = WT  # Add the WT sequence for each substitution
else:
    # Create an empty DataFrame with the same columns as deletions_df if needed
    singles_df = pd.DataFrame(columns=['ID', 'Sequence', 'Exon_id', 'Chromosome', 'WT_sequence'])

    # Generate all deletions
if isinstance(deletion_sizes, str):
    deletion_sizes = [int(size) for size in deletion_sizes.split(',')]

deletions_list = []


for each_del_size in deletion_sizes:
    # Ensure each_del_size is an integer
    each_del_size_int = int(each_del_size)  # Convert to integer if not already
    variants_with_kmer_deletion = generate_all_deletions(each_del_size_int, WT, exon_id, chromosome, padded_chrom_start)
    for variant_id, variant_seq in zip(variants_with_kmer_deletion['ID'], variants_with_kmer_deletion['Sequence']):
        deletions_list.append({'ID': variant_id, 'Sequence': variant_seq, 'Exon_id': exon_id, 'Chromosome': chromosome, 'WT_sequence': WT})

    # Generate all deletions})
    deletions_df = pd.DataFrame(deletions_list)

    # Combine substitutions and deletions for this exon
    exon_variants_df = pd.concat([singles_df, deletions_df], ignore_index=True)

    # Concatenate the results for all exons
    combined_variants_df = pd.concat([combined_variants_df, exon_variants_df], ignore_index=True)

# Remove duplicated sequences and combine IDs
combined_variants_df = remove_duplicates_in_variant_data_frame(combined_variants_df)

# Save the combined DataFrame to a CSV file
combined_variants_df.to_csv("combined_variants_data.csv", index=False)
def extract_individual_variants_info(variants_df, variant_type='_Del_'):
    # Initialize lists to store extracted information
    CHROM, POS, ID, REF, ALT, WT_SEQ = [], [], [], [], [], []

    # Filter rows based on variant type
    filtered_variants_df = variants_df[variants_df['ID'].str.contains(variant_type, regex=False)]

    # Iterate over each row in the filtered DataFrame
    for index, row in filtered_variants_df.iterrows():
        # Split ID if it contains multiple variants joined together
        ids = row['ID'].split(';')
        for individual_id in ids:
            WT_sequence = row['WT_sequence']
            # Ensure chrom_info extraction is robust
            chrom_match = re.search(r"chr(\d+|\w+)", individual_id)  # Adjusted to match more than just numbers, e.g., X, Y chromosomes
            if chrom_match:
                chrom_info = chrom_match.group(1)
                padded_chrom_start = int(individual_id.split(':')[1].split('_')[0])

                if variant_type == '_Del_':
                    # Process deletions
                    del_info = re.search(r"_Del_k(\d+)_(\d+)-(\d+)", individual_id)
                    if del_info:
                        deletion_start_pos, deletion_end_pos = int(del_info.group(2)), int(del_info.group(3))
                        start_pos_abs = padded_chrom_start + deletion_start_pos - 2
                        REF_seq = WT_sequence[deletion_start_pos - 2:deletion_end_pos] if deletion_start_pos - 2 < len(WT_sequence) else "."
                        ALT_value = REF_seq[0] if REF_seq else "."
                else:
                    # Process substitutions
                    sub_info = re.search(r"_Sub_([ATGC])(\d+)([ATGC])", individual_id)
                    if sub_info:
                        ref_nucleotide, position_of_substitution, alt_nucleotide = sub_info.group(1), int(sub_info.group(2)), sub_info.group(3)
                        start_pos_abs = padded_chrom_start + position_of_substitution - 1
                        REF_seq, ALT_value = ref_nucleotide, alt_nucleotide

                # Append to lists
                CHROM.append(chrom_info)  # Keep as string to accommodate 'X', 'Y', etc.
                POS.append(start_pos_abs)
                ID.append(individual_id)
                REF.append(REF_seq)
                ALT.append(ALT_value)
                WT_SEQ.append(WT_sequence)

    # Create the new DataFrame based on variant type
    info_df = pd.DataFrame({
        'CHROM': CHROM,
        'POS': [int(pos) for pos in POS],
        'ID': ID,
        'REF': REF,
        'ALT': ALT,
        'WT_sequence': WT_SEQ
    })

    return info_df

  # Usage
deletion_info_df = extract_individual_variants_info(combined_variants_df, variant_type='_Del_')
substitution_info_df = extract_individual_variants_info(combined_variants_df, variant_type='_Sub_')


# Combine deletion and substitution information into one DataFrame
combined_vcf_info_df = pd.concat([deletion_info_df, substitution_info_df], ignore_index=True)
# After constructing or modifying the DataFrame
combined_vcf_info_df['POS'] = combined_vcf_info_df['POS'].astype(int)

# Filter out specific rows if necessary (e.g., based on REF values)
# This step is optional and depends on your specific requirements
# Filter out rows with empty or whitespace-only values in the 'REF' column
filtered_vcf_info_df = combined_vcf_info_df[combined_vcf_info_df['REF'].str.strip() != ""]
filtered_vcf_info_df = filtered_vcf_info_df.drop_duplicates()

# Define the VCF header
vcf_header = """##fileformat=VCFv4.2
##reference=GRCh38/hg38
##contig=<ID=chr1,length=249250621>
##contig=<ID=chr2,length=243199373>
##contig=<ID=chr3,length=198022430>
##contig=<ID=chr4,length=191154276>
##contig=<ID=chr5,length=180915260>
##contig=<ID=chr6,length=171115067>
##contig=<ID=chr7,length=159138663>
##contig=<ID=chr8,length=146364022>
##contig=<ID=chr9,length=141213431>
##contig=<ID=chr10,length=135534747>
##contig=<ID=chr11,length=135006516>
##contig=<ID=chr12,length=133851895>
##contig=<ID=chr13,length=115169878>
##contig=<ID=chr14,length=107349540>
##contig=<ID=chr15,length=102531392>
##contig=<ID=chr16,length=90354753>
##contig=<ID=chr17,length=81195210>
##contig=<ID=chr18,length=78077248>
##contig=<ID=chr19,length=59128983>
##contig=<ID=chr20,length=63025520>
##contig=<ID=chr21,length=48129895>
##contig=<ID=chr22,length=51304566>
##contig=<ID=chrX,length=155270560>
##contig=<ID=chrY,length=59373566>
#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO"""

# Format each row into a VCF entry
vcf_rows = []
for _, row in filtered_vcf_info_df.iterrows():
    vcf_entry = f"chr{row['CHROM']}\t{row['POS']}\t{row['ID']}\t{row['REF']}\t{row['ALT']}\t30\tPASS\t."
    vcf_rows.append(vcf_entry)
# Filter VCF rows to exclude REF = "t" where there is missing reference base
vcf_rows = [row for row in vcf_rows if not row.endswith("\t\t.\t30\tPASS\t.")]
# Define the filename for the VCF file
vcf_filename = f"{ensembl_gene_name_query}_{exon_number_query}_{ensembl_exon_id}_combined_variants.vcf"

# Write the VCF file
with open(vcf_filename, 'w') as vcf_file:
    vcf_file.write(vcf_header + '\n')
    vcf_file.writelines('\n'.join(vcf_rows))

print(f"VCF file '{vcf_filename}' has been created.")


In [ ]:
# @title Run SpliceAI and save output on machine
output_filename = f"{ensembl_gene_name_query}_{exon_number_query}_{ensembl_exon_id}_3prime{expand_3prime}_5prime{expand_5prime}_ntw{window}_spliceai_output.vcf"
!spliceai -I {vcf_filename} -O {output_filename} -R hg{hg}.fa -A grch{GRCh} -D {window}
if automatic_download == True :
     from google.colab import files
     files.download(output_filename)
     files.download(query_csv_file_path)
spliceAI_table = pd.read_csv(output_filename, skiprows=28, sep='\t') ##chnage to name of output
spliceAI_table = spliceAI_table.drop_duplicates()

In [ ]:
# @title Generate triangular heatmap of all in silico deletions
import pandas as pd
def extract_max_delta_score(info):
    # Extract the SpliceAI scores part from the INFO column
    spliceai_scores = info.split(';')[0]  # Assuming SpliceAI scores are the first in the INFO field
    delta_scores = spliceai_scores.split('|')[2:6]
    delta_scores = [float(score) for score in delta_scores]

    # Check if all scores are zero
    if all(score == 0 for score in delta_scores):
        return 0

    # Find the maximum delta score
    max_score = max(delta_scores)

    # Find the index of the first occurrence of the maximum score
    max_score_index = delta_scores.index(max_score)

    # Adjust the result based on the index
    result = delta_scores[max_score_index]
    if max_score_index in [1, 3]:
        result *= -1

    return result

def extract_exon_id(id_string):
    # Split the string by underscores
    parts = id_string.split('_')
    # Check if there are enough parts to contain an exon ID
    if len(parts) > 1:
        # Remove the "ex" prefix and return the exon ID
        exon_id = parts[1][2:] if parts[1].startswith("ex") else parts[1]
        return exon_id
    else:
        # Return None if no exon ID is found
        return None

spliceAI_table['predictions'] = spliceAI_table['INFO'].apply(extract_max_delta_score)
# Assuming spliceAI_table is a pandas DataFrame with an 'ID' column
spliceAI_table['exon_id'] = spliceAI_table['ID'].apply(extract_exon_id)


vector_starts = []
vector_ends = []
vector_delta_psi = []

# Extract information from IDs in the spliceAI_table
for index, row in spliceAI_table.iterrows():
    id_info = row['ID']
    # Check if ID format matches deletion pattern
    match = re.search(r"Del_k(\d+)_(\d+)-(\d+)", id_info)
    if match:
        # Extract deletion size and start position from the ID
        deletion_size = int(match.group(1))
        start_position = int(match.group(2))
        end_position = int(match.group(3))
        # Append extracted information to lists
        vector_starts.append(start_position)
        vector_ends.append(end_position)
        vector_delta_psi.append(row['predictions'])

# Prepare deletion heatmap data
heatmap_df = pd.DataFrame({
    'start': vector_starts,
    'end': vector_ends,
    'dPSI': vector_delta_psi
})
heatmap_df

# Create custom colormap and set color limits
my_colours = sns.color_palette("Spectral_r", 11)
cmap = LinearSegmentedColormap.from_list("my_colormap", my_colours)

# Create heatmap with flipped data
plt.figure(figsize=(12, 12))

# Flip the data by transposing the DataFrame
flipped_df = heatmap_df.pivot("start", "end", "dPSI")

# Set x-axis ticks at every 5th number
x_ticks = np.arange(0, len(flipped_df.columns), 5)
#ax = sns.heatmap(flipped_df, cmap=cmap, cbar_kws={'label': 'Max (Splice AI delta score)', 'location': 'left'}, vmin=-0.5, vmax=0.25, xticklabels=x_ticks, yticklabels=True)


# Assuming flipped_df is your DataFrame and it contains a column named 'vector_delta_psi'
min_delta_psi = heatmap_df['dPSI'].min()
max_delta_psi = heatmap_df['dPSI'].max()

# Create the heatmap
ax = sns.heatmap(flipped_df, cmap=cmap, cbar_kws={'label': 'Max (Splice AI delta score)', 'location': 'left'}, vmin=min_delta_psi, vmax=max_delta_psi, xticklabels=x_ticks, yticklabels=True)
# Assuming position_lookup already includes an 'exon_id' column
#position_to_label = all_position_lookups.apply(lambda x: ((x['exon ID'],x['position']), f"{x['chromosome']}:{x['reference position']}:{x['nucleotide']}"), axis=1).to_dict()
# Create the position_to_label dictionary starting at 1 for keys
#position_to_label = all_position_lookups.apply(lambda x: ((x['exon ID'], x['position']), f"{x['chromosome']}:{x['reference position']}:{x['nucleotide']}"), axis=1).to_dict()
position_to_label = all_position_lookups.apply(
    lambda x: ((x['exon ID'], x['position']), f"{x['chromosome']}:{x['reference position']}:{x['nucleotide']} ({x['region']})"),
    axis=1
).to_dict()


#position_to_label = all_position_lookups.apply(lambda x: ((x['exon ID'], x['position']), f"{x['chromosome']}:{x['reference position']}:{x['nucleotide']} ({x['region']})"), axis=1).to_dict()
# Assuming all_position_lookups is a DataFrame with columns 'exon ID', 'position', 'chromosome', 'reference position', 'nucleotide'

# Update x-axis ticks with labels from the position_lookup mapping
# Example usage of set_xticklabels with a dictionary that has tuple keys
ax.set_xticklabels([position_to_label.get(( pos), '') for pos in flipped_df.columns[x_ticks]], rotation=90)


# Set y-axis ticks at every 5th number
y_ticks = np.arange(0, len(flipped_df.index), 5)
ax.set_yticks(y_ticks)
ax.set_yticklabels([position_to_label.get(pos, '') for pos in flipped_df.index[y_ticks]], rotation=0)


# Assuming position_lookup is your DataFrame with columns 'position', 'chromosome', 'reference_position', 'nucleotide'
# Create a mapping from position to formatted string


# Create a secondary x-axis at the top
ax2 = ax.secondary_xaxis('top')
ax2.set_xticks(x_ticks)
# Example usage of set_xticklabels with a dictionary that has tuple keys
#ax2.set_xticklabels([(lambda x: position_to_label.get(x, '')[1])(x) for x in x_ticks], rotation=90)
ax2.set_xticklabels([(lambda x: position_to_label.get(x, '')[1] if position_to_label.get(x, '') else '')(x) for x in x_ticks], rotation=90)

ax3 = ax.secondary_yaxis('right')
ax3.set_yticks(y_ticks)

ax3.set_yticklabels([(lambda y: position_to_label.get(y, '')[1] if position_to_label.get(y, '') else '')(y) for y in y_ticks])

#ax3.set_yticklabels([(lambda y: position_to_label.get(y, '')[1])(y) for y in y_ticks])

# Remove the bottom x-axis
ax.set_xlabel('')
ax.set_xticklabels([])

# Remove the left-hand side y-axis
ax.set_ylabel('')
ax.set_yticklabels([])

# Set axis labels for the right y-axis and top x-axis
ax3.set_ylabel("First Deleted Position")
ax2.set_xlabel("Last Deleted Position")
plt.title(f"In silico DIM with spliceAI, all deletions, - Gene: {ensembl_gene_name_query}, Exon Number: {exon_number_query}, nt window :{window}_heatmap.pdf")


# Save heatmap

filename_heatmap = f"{ensembl_gene_name_query}_Exon_{exon_number_query}_{ensembl_exon_id}_ntw_{window}_heatmap.pdf"
plt.savefig(filename_heatmap, bbox_inches='tight')
if automatic_download == True :
     from google.colab import files
     files.download(filename_heatmap)
plt.show()


In [ ]:
# @title Generate plot spliceAI predictions for 1-6nt deletions with fitted LOESS curve to exonic region
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm


# Initialize lists
spliceAI_vector = []
position_vector = []
k_vector = []

# Determine the length of WT
wt_length = len(WT)  # Replace with actual WT length

# Sliding window size 1-10
for k in range(1, 11):
    for p in range(1, wt_length - k + 1):
        this_id = f"Del_k{k}_{p}-{p + k - 1}"
        idx = spliceAI_table[spliceAI_table['ID'].str.contains(this_id)].index

        if len(idx) > 0:
            spliceAI_vector.extend(spliceAI_table.loc[idx, 'predictions'])
            position_vector.extend([p + (k / 2) - 0.5] * len(idx))
            k_vector.extend([k] * len(idx))

# Create DataFrame
plot_df = pd.DataFrame({
    'spliceAI': spliceAI_vector,
    'k': k_vector,
    'position': position_vector
})

# Filter for k in range 1-6
plot_df_filtered = plot_df[plot_df['k'].isin(range(1, 7))]

# Define color palette
my_shapes = ['o', 's', 'D', '^', 'v', 'P']

# Filter the DataFrame for fitting the LOESS curve TO BE FINALISED
plot_df_loess = plot_df_filtered[(plot_df_filtered['position'] >= expand_5prime) & (plot_df_filtered['position'] <= wt_length - expand_3prime)]

# Plotting
plt.figure(figsize=(15, 10))

# Add lighter grey shading for potential silencer regions
lowess = sm.nonparametric.lowess(plot_df_loess['spliceAI'], plot_df_loess['position'], frac=0.15)
lowess_x, lowess_y = zip(*lowess)
for x, y in zip(lowess_x, lowess_y):
    if y > 0:
        plt.axvspan(x - 0.5, x + 0.5, color='lightgray', alpha=0.3)

# Plot individual points with different shapes for all data
for i, k in enumerate(range(1, 7)):
    data = plot_df_filtered[plot_df_filtered['k'] == k]
    sns.scatterplot(data=data, x='position', y='spliceAI', marker=my_shapes[i], label=f'Deletion Length {k} nt')

# Plot LOESS curve in a darker golden color
plt.plot(lowess_x, lowess_y, color='#FFA500', linewidth=2, label='LOESS Curve')

# Compute and plot the 95% confidence interval for LOESS
confidence = 1.96 * np.std(plot_df_loess['spliceAI']) / np.sqrt(len(plot_df_loess['spliceAI']))
plt.fill_between(lowess_x, lowess_y - confidence, lowess_y + confidence, color='#FFA500', alpha=0.5)

# Add a horizontal line at y=0
plt.axhline(0, color='gray', linestyle='--', linewidth=1)

# Use the position-to-label mapping for x-axis ticks
x_ticks = np.arange(min(plot_df['position']), max(plot_df['position']) + 1, step=2)
x_tick_labels = ([(lambda x: position_to_label.get(x, '')[1])(x) for x in x_ticks])
plt.xticks(x_ticks, x_tick_labels, rotation=90)

# Add a dummy plot entry for "Silencer" and "Enhancer" in legend
plt.scatter([], [], color='lightgray', alpha=0.3, label='Silencer')
plt.scatter([], [], color='white', label='Enhancer')

plt.legend(loc='upper left')

# Adding Ensembl Gene Name and Exon Number to the title
plt.title(f"In silico DIM with spliceAI, fitted Loess to 1-6nt deletions, - Gene: {ensembl_gene_name_query}, Exon Number: {exon_number_query}, Ensembl exon ID : {ensembl_exon_id} nt window : {window}")

# Saving and showing the plot
filename_plot = f"{ensembl_gene_name_query}_Exon_{exon_number_query}_{ensembl_exon_id}_ntw_{window}_loess_plot.pdf"
plt.savefig(filename_plot, bbox_inches='tight')
if automatic_download == True :
     from google.colab import files
     files.download(filename_plot)
plt.show()

In [ ]:
# @title Plot longitudinal heatmap of deletions and substitutions
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

##need to define sub_df
# Assuming spliceAI_table is already loaded and has 'predictions' column calculated
# Initialize lists
vector_starts = []
deletion_sizes = []
vector_delta_psi = []

# Extract information from IDs in the spliceAI_table
for index, row in spliceAI_table.iterrows():
    id_info = row['ID']
    # Check if ID format matches deletion pattern
    match = re.search(r"Del_k(\d+)_(\d+)-(\d+)", id_info)
    if match:
        # Extract deletion size and start position from the ID
        deletion_size = int(match.group(1))
        start_position = int(match.group(2))

        # Append extracted information to lists
        vector_starts.append(start_position)
        deletion_sizes.append(deletion_size)
        vector_delta_psi.append(row['predictions'])

# Prepare deletion heatmap data
del_heatmap_df = pd.DataFrame({
    'start': vector_starts,
    'deletion_size': deletion_sizes,
    'dPSI': vector_delta_psi
})
# Transpose the pivot for deletions to have deletion size on x-axis and start position on y-axis
del_pivot_df = del_heatmap_df.pivot("deletion_size", "start", "dPSI")

# Initialize lists to store parsed data
positions = []
refs = []
alts = []
predictions = []

# Regex pattern to extract position, REF, and ALT from ID
pattern = re.compile(r"Sub_([ATCG])(\d+)([ATCG])")

for idx, row in spliceAI_table.iterrows():
    match = pattern.search(row['ID'])
    if match:
        ref, pos, alt = match.groups()
        positions.append(int(pos))
        refs.append(ref)
        alts.append(alt)
        predictions.append(row['predictions'])  # Assuming 'predictions' column exists

# Create DataFrame
sub_df = pd.DataFrame({
    'Position': positions,
    'REF': refs,
    'ALT': alts,
    'Prediction': predictions
})

# Prepare substitution heatmap data
sub_df_filtered = sub_df[sub_df['Position'] != 1]
sub_pivot_df = sub_df_filtered.pivot_table(index='ALT', columns='Position', values='Prediction', aggfunc='mean')

# Color scale limits
vmin, vmax = -0.5, 0.5  # Specified color scale limits

# Combine both heatmaps vertically with a shared color scale
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(20, 12), sharex=True, gridspec_kw={'height_ratios': [1, 3]})

# Plot deletion heatmap
sns.heatmap(del_pivot_df, cmap="Spectral_r", cbar=True, ax=ax1, vmin=vmin, vmax=vmax, cbar_kws={'label': 'SpliceAI Prediction'})
ax1.set_title("Deletions")
ax1.set_xlabel("")
ax1.set_ylabel("Deletion Size (nt)")

# Plot substitution heatmap
sns.heatmap(sub_pivot_df, cmap="Spectral_r", cbar=True, ax=ax2, vmin=vmin, vmax=vmax, cbar_kws={'label': 'SpliceAI Prediction'}, annot=False)
ax2.set_title("Substitutions")
ax2.set_xlabel("Position")
ax2.set_ylabel("ALT Base")

# Adjust x-axis ticks to shift to the right by 0.5
x_ticks = np.arange(0.5, len(sub_pivot_df.columns) + 0.5, 1)  # Shifting ticks to the right by 0.5
x_tick_labels = [position_to_label.get(x+1, '') for x in range(len(sub_pivot_df.columns))]  # Adjusting for shifted ticks

ax2.set_xticks(x_ticks)
ax2.set_xticklabels(x_tick_labels, rotation=90)
# Annotate NaN values with asterisks in substitution heatmap
for ytick, row in enumerate(sub_pivot_df.itertuples(index=False)):
    for xtick, value in enumerate(row):
        if pd.isna(value):  # Check for NaN
            ax2.text(xtick + 0.5, ytick + 0.5, '*', ha='center', va='center', color='black')  # Adjusted position for annotation



plt.tight_layout()
filename_heatmap_long = f"{ensembl_gene_name_query}_Exon_{exon_number_query}_{ensembl_exon_id}_ntw_{window}_heatmap_long.pdf"
plt.title(f"In silico DIM with spliceAI, all deletions & substitutions, - Gene: {ensembl_gene_name_query}, Exon Number: {exon_number_query}, Ensembl exon ID : {ensembl_exon_id} nt window :{window}_heatmap_long")
plt.savefig(filename_heatmap_long, bbox_inches='tight')
if automatic_download == True :
     from google.colab import files
     files.download(filename_heatmap_long)
plt.show()

